In [ ]:
# install huggingface and datasets
!pip install -q datasets transformers
!pip install torch
!pip install pandas
!pip install pyarrow

In [ ]:
from datasets import load_dataset, ClassLabel
from transformers import BertForTokenClassification, BertTokenizer, TrainingArguments, Trainer
import numpy as np
import torch
from sklearn import metrics


import pandas as pd
import datasets
from datasets.features import ClassLabel

In [ ]:
torch.cuda.is_available()
device = torch.device("cuda")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/MyDrive/e_ML4NLP2/v2.1/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ajmc  hipe2020	letemps  newseye  sonar  topres19th


In [ ]:
train_path = '/content/drive/MyDrive/e_ML4NLP2/v2.1/hipe2020/fr/HIPE-2022-v2.1-hipe2020-train-fr.tsv'
dev_path = '/content/drive/MyDrive/e_ML4NLP2/v2.1/hipe2020/fr/HIPE-2022-v2.1-hipe2020-dev-fr.tsv'
test_path = '/content/drive/MyDrive/e_ML4NLP2/v2.1/hipe2020/fr/HIPE-2022-v2.1-hipe2020-test_allmasked-fr.tsv'

model = 'dbmdz/bert-base-historic-multilingual-cased'
#model = 'setu4993/LaBSE'
#model = 'dbmdz/bert-base-french-europeana-cased'
#model = 'dbmdz/electra-base-french-europeana-cased-discriminator'

In [ ]:
# import dataset from cloned git repo
def load_dataset(path):
    df = pd.read_csv(path, sep='\t', skip_blank_lines=False, engine='python', quoting=3)
    # error_bad_lines=False, 
    return df


tsv_train = load_dataset(train_path)
tsv_dev = load_dataset(dev_path)
tsv_test = load_dataset(test_path)

In [ ]:
def simple_preprocess(dataframe):
    # Add end_of_document token in df
    dataframe = dataframe.dropna(subset=['TOKEN'])

    # Filter out metadata rows beginning with #
    dataframe = dataframe[~dataframe['TOKEN'].astype(str).str.startswith('#')]
    dataframe = dataframe[~dataframe['TOKEN'].astype(str).str.startswith('\t')]

    #transforming nan var from Float to string to use in (***)
    dataframe.MISC = dataframe.MISC.fillna('')

    return dataframe

In [ ]:
tsv_train = simple_preprocess(tsv_train)
tsv_dev = simple_preprocess(tsv_dev)
tsv_test = simple_preprocess(tsv_test)
#tsv_train = tsv_train.reset_index()
tsv_dev.head(100)

,TOKEN,NE-COARSE-LIT,NE-COARSE-METO,NE-FINE-LIT,NE-FINE-METO,NE-FINE-COMP,NE-NESTED,NEL-LIT,NEL-METO,MISC
11,FAITS,O,O,O,O,O,O,_,_,_
12,DIVERS,O,O,O,O,O,O,_,_,EndOfLine
13,La,O,O,O,O,O,O,_,_,_
14,panique,O,O,O,O,O,O,_,_,_
15,des,O,O,O,O,O,O,_,_,_
...,...,...,...,...,...,...,...,...,...,...
106,.,O,O,O,O,O,O,_,_,EndOfSentence
107,Six,O,O,O,O,O,O,_,_,_
108,d,O,O,O,O,O,O,_,_,NoSpaceAfter
109,',O,O,O,O,O,O,_,_,EndOfLine|NoSpaceAfter


In [ ]:
label_set = tsv_train['NE-COARSE-LIT'].unique()
# define the label mapping for NER
label_list = label_set.tolist()
label_list.append('_')
label_num = len(label_list)

labels = ClassLabel(num_classes=label_num, names=label_list)

labels

ClassLabel(num_classes=12, names=['O', 'B-time', 'I-time', 'B-org', 'I-org', 'B-loc', 'I-loc', 'B-pers', 'I-pers', 'B-prod', 'I-prod', '_'], id=None)

In [ ]:
def create_huggingface_file(dataframe):
    import pyarrow as pa
    import re
    from datasets import Dataset

    #creating dataset in json
    hug_out = []
    idx = 0 
    items = {'id': idx,'words':[ ], 'ner': [ ]}
    hug_out.append(items)
    for index, row in dataframe.iterrows():
        if  not re.search(r'EndOfSentence', row['MISC']):
            items['words'].append(row['TOKEN'])
            items['ner'].append(labels.str2int(row['NE-COARSE-LIT']))
        else:
            items['words'].append(row['TOKEN'])
            items['ner'].append(labels.str2int(row['NE-COARSE-LIT']))
            idx += 1
            items = {'id': idx,'words':[ ], 'ner': [ ]}
            hug_out.append(items)
    #filter hug_out out, delete items which has len(words) > 380
    #hug_out = filter(lambda x: len(x['words']) < 380, hug_out)
    #json to df
    hug_out = pd.DataFrame(hug_out)

    # delete all sentences that are too long
    #hug_out = hug_out[hug_out['words'].map(len) < 512] #why does not work? QA

    ### convert to Huggingface dataset
    hug_out = Dataset(pa.Table.from_pandas(hug_out))

    return hug_out

In [ ]:
train = create_huggingface_file(tsv_train)
val = create_huggingface_file(tsv_dev)
test = create_huggingface_file(tsv_test)

#look at training data
for i in range(10):
  print(train[i])

print(len(train))
print(len(val))
print(len(test))

{'id': 0, 'words': ['NOUVELLES', 'SUISSES', '—', 'En', '1887', ',', 'la', 'Société', 'suisse', 'du', 'Grutli', 's', "'", 'est', 'accrue', 'de', '40', 'sections', ';', 'l', "'", 'association', 'compte', 'actuellement', '12,000', 'membres', '.'], 'ner': [0, 0, 0, 1, 2, 0, 0, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'id': 1, 'words': ['Environ', '30,000', 'fr', '.', 'ont', 'été', 'versés', 'dans', 'la', 'caisse', 'créée', 'pour', 'subventionner', 'les', 'grèves', '.'], 'ner': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
{'id': 2, 'words': ['Ecoles', 'de', 'fonctionnaires', '.'], 'ner': [0, 0, 0, 0]}
{'id': 3, 'words': ['—', 'On', 'parle', 'de', 'fonder', 'deux', 'facultés', 'de', 'sciences', 'politiques', ',', 'l', "'", 'une', 'à', 'Berne', ',', 'l', "'", 'autre', 'à', 'Genève', 'ou', 'à', 'Lausanne', ',', 'afin', 'de', 'donner', 'ainsi', 'aux', 'aspirants', 'au', 'service', 'fédéral', ',', 'l', "'", 'occasion', 'd', "'", 'acquérir', 'les', 'connaissances', 'qui'

In [ ]:
'''LOADERSCRIPT = train_path = '/content/drive/MyDrive/e_ML4NLP2/dataloader.py'
MODEL = "dbmdz/bert-base-german-europeana-cased"'''

'LOADERSCRIPT = train_path = \'/content/drive/MyDrive/e_ML4NLP2/dataloader.py\'\nMODEL = "dbmdz/bert-base-german-europeana-cased"'

In [ ]:
'''de_total = load_dataset(LOADERSCRIPT, "de")'''

'de_total = load_dataset(LOADERSCRIPT, "de")'

In [ ]:
'''train = de_total["train"]
val = de_total["validation"]
test = de_total["test"]'''

'train = de_total["train"]\nval = de_total["validation"]\ntest = de_total["test"]'

In [ ]:
'''tiny_shuffle_train =  train.shuffle(seed=42).select(range(100))
tiny_train = tiny_shuffle_train.select(range(0,80))
tiny_shuffle_test = test.shuffle(seed=42).select(range(100))
tiny_test = tiny_shuffle_test.select(range(80,100))'''

'tiny_shuffle_train =  train.shuffle(seed=42).select(range(100))\ntiny_train = tiny_shuffle_train.select(range(0,80))\ntiny_shuffle_test = test.shuffle(seed=42).select(range(100))\ntiny_test = tiny_shuffle_test.select(range(80,100))'

In [ ]:
'''tiny_shuffle_train[10]'''

'tiny_shuffle_train[10]'

In [ ]:
'''labels = train.info.features["NE_COARSE_LIT"].feature
labels'''

'labels = train.info.features["NE_COARSE_LIT"].feature\nlabels'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(model)

https://huggingface.co/dbmdz/bert-base-historic-multilingual-cased/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpkr2usyyv


Downloading:   0%|          | 0.00/207k [00:00<?, ?B/s]

storing https://huggingface.co/dbmdz/bert-base-historic-multilingual-cased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/d7256fb2f051c3bb10f0ab260557cbcd5101338e37d9ed235027f9e8d5bd826f.e873d4342a4099c85582b4730d903356893a02b9efbb8392416275ea2d014b16
creating metadata file for /root/.cache/huggingface/transformers/d7256fb2f051c3bb10f0ab260557cbcd5101338e37d9ed235027f9e8d5bd826f.e873d4342a4099c85582b4730d903356893a02b9efbb8392416275ea2d014b16
https://huggingface.co/dbmdz/bert-base-historic-multilingual-cased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmps3p44ccf


Downloading:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

storing https://huggingface.co/dbmdz/bert-base-historic-multilingual-cased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/9d686978c59c347e61d3c82af6602bef1552cb8e11d906ab5fd21f451dddb133.bd85f6a0d9c2848260fdda7056a557a7fa213a57b5ed27b530abd20572c0cbf4
creating metadata file for /root/.cache/huggingface/transformers/9d686978c59c347e61d3c82af6602bef1552cb8e11d906ab5fd21f451dddb133.bd85f6a0d9c2848260fdda7056a557a7fa213a57b5ed27b530abd20572c0cbf4
loading file https://huggingface.co/dbmdz/bert-base-historic-multilingual-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/d7256fb2f051c3bb10f0ab260557cbcd5101338e37d9ed235027f9e8d5bd826f.e873d4342a4099c85582b4730d903356893a02b9efbb8392416275ea2d014b16
loading file https://huggingface.co/dbmdz/bert-base-historic-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/dbmdz/bert-base-historic-multilingual-cased/resolve/main/special

Downloading:   0%|          | 0.00/561 [00:00<?, ?B/s]

storing https://huggingface.co/dbmdz/bert-base-historic-multilingual-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/5ce7e041308a3a4d2a07e1655ad227e92ce92724a53cbba35dabc5870332286f.52c144249cc94a2c4412d3bb497665d3d77c1389cfc2bdce33b2e6a8d3220542
creating metadata file for /root/.cache/huggingface/transformers/5ce7e041308a3a4d2a07e1655ad227e92ce92724a53cbba35dabc5870332286f.52c144249cc94a2c4412d3bb497665d3d77c1389cfc2bdce33b2e6a8d3220542
loading configuration file https://huggingface.co/dbmdz/bert-base-historic-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/5ce7e041308a3a4d2a07e1655ad227e92ce92724a53cbba35dabc5870332286f.52c144249cc94a2c4412d3bb497665d3d77c1389cfc2bdce33b2e6a8d3220542
Model config BertConfig {
  "_name_or_path": "dbmdz/bert-base-historic-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act":

In [ ]:
train = train.filter(lambda example, idx: idx != len(train)-1, with_indices=True)
val = val.filter(lambda example, idx: idx != len(val)-1, with_indices=True)
test = test.filter(lambda example, idx: idx != len(test)-1, with_indices=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
def tokenize_func(batch):
    tokenized = tokenizer(batch["words"], is_split_into_words=True, padding="max_length", max_length=100, truncation=True)
    tokenized_words = [
        [
            tokenizer(word, add_special_tokens=False)["input_ids"]
            for word in sent
        ]
        for sent in batch["words"]
    ]
    tokenized_ner = []

    for sent_words, sent_nes in zip(tokenized_words, batch["ner"]):
        tokenized_ner.append(
            [labels.str2int("O")]  # BOS symbol
            + [
                ne
                for subwords, ne in zip(sent_words, sent_nes)
                for _ in range(len(subwords))
            ]
            + [labels.str2int("O")]  # EOS symbol
        )

    # Padding with "O"
    tokenized["labels"] = [(ner + [labels.str2int("O")] * (100 - len(ner)))[:100] for ner in tokenized_ner]
    tokenized["subwords"] = tokenized_words
    return tokenized

def tokenize_nolabel_func(batch):
    tokenized = tokenizer(batch["words"], is_split_into_words=True, padding="max_length", max_length=100, truncation=True)
    tokenized_words = [
        [
            tokenizer(word, add_special_tokens=False)["input_ids"]
            for word in sent
        ]
        for sent in batch["words"]
    ]

    tokenized["subwords"] = tokenized_words
    return tokenized

#tiny_train_tokenized = tiny_train.map(tokenize_func, batched=True, batch_size=50)
#tiny_test_tokenized = tiny_test.map(tokenize_nolabel_func, batched=True, batch_size=50)

train_tokenized = train.map(tokenize_func, batched=True, batch_size=50)
val_tokenized = val.map(tokenize_func, batched=True, batch_size=50)
test_tokenized = test.map(tokenize_nolabel_func, batched=True, batch_size=50)


  0%|          | 0/115 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/30 [00:00<?, ?ba/s]

In [ ]:
'''# train for 1 epochs, evaluate after each epoch
training_args = TrainingArguments(
    "test_trainer",
    num_train_epochs=3,
    evaluation_strategy="epoch"
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
)
trainer.train()'''

'# train for 1 epochs, evaluate after each epoch\ntraining_args = TrainingArguments(\n    "test_trainer",\n    num_train_epochs=3,\n    evaluation_strategy="epoch"\n)\n\n\ntrainer = Trainer(\n    model=model,\n    args=training_args,\n    train_dataset=tokenized_train,\n    eval_dataset=tokenized_dev,\n)\ntrainer.train()'

In [ ]:
training_args = TrainingArguments(
    output_dir = "content/drive/MyDrive/e_ML4NLP2/"
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
model = BertForTokenClassification.from_pretrained(model, num_labels=labels.num_classes)
trainer = Trainer(model=model, args=training_args, train_dataset=train_tokenized)
trainer.train()

loading configuration file https://huggingface.co/dbmdz/bert-base-historic-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/5ce7e041308a3a4d2a07e1655ad227e92ce92724a53cbba35dabc5870332286f.52c144249cc94a2c4412d3bb497665d3d77c1389cfc2bdce33b2e6a8d3220542
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

storing https://huggingface.co/dbmdz/bert-base-historic-multilingual-cased/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/20544e121926f3db8808196e9fe1f2e2c41db9d94994284f8fe4fbcba0e2d7b8.157daaa628a92269ef54709629afd1598947ac3f2b5a3d3c7363896ff0928d2f
creating metadata file for /root/.cache/huggingface/transformers/20544e121926f3db8808196e9fe1f2e2c41db9d94994284f8fe4fbcba0e2d7b8.157daaa628a92269ef54709629afd1598947ac3f2b5a3d3c7363896ff0928d2f
loading weights file https://huggingface.co/dbmdz/bert-base-historic-multilingual-cased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/20544e121926f3db8808196e9fe1f2e2c41db9d94994284f8fe4fbcba0e2d7b8.157daaa628a92269ef54709629afd1598947ac3f2b5a3d3c7363896ff0928d2f
Some weights of the model checkpoint at dbmdz/bert-base-historic-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predict

Step,Training Loss
500,0.106500
1000,0.038000
1500,0.025100
2000,0.012700


Saving model checkpoint to content/drive/MyDrive/e_ML4NLP2/checkpoint-500
Configuration saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-500/config.json
Model weights saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-500/pytorch_model.bin
Saving model checkpoint to content/drive/MyDrive/e_ML4NLP2/checkpoint-1000
Configuration saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-1000/config.json
Model weights saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to content/drive/MyDrive/e_ML4NLP2/checkpoint-1500
Configuration saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-1500/config.json
Model weights saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to content/drive/MyDrive/e_ML4NLP2/checkpoint-2000
Configuration saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-2000/config.json
Model weights saved in content/drive/MyDrive/e_ML4NLP2/checkpoint-2000/pytorch_model.bin


Training comp

TrainOutput(global_step=2154, training_loss=0.04307519869330644, metrics={'train_runtime': 751.9247, 'train_samples_per_second': 22.913, 'train_steps_per_second': 2.865, 'total_flos': 879353287927200.0, 'train_loss': 0.04307519869330644, 'epoch': 3.0})

In [ ]:
predictions = trainer.predict(test_tokenized)

The following columns in the test set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner, words, subwords, id. If ner, words, subwords, id are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1462
  Batch size = 8


In [ ]:
outfile = open("/content/drive/MyDrive/e_ML4NLP2/teamname_bundle4_de_0.tsv", "w")
outfile.write("TOKEN\tNE-COARSE-LIT\tNE-COARSE-METO\tNE-FINE-LIT\tNE-FINE-METO\tNE-FINE-COMP\tNE-NESTED\tNEL-LIT\tNEL-METO\tMISC")

for pred, tinput in zip(predictions.predictions, test_tokenized):
    pred_word_labels = []

    try:
        sent_len = tinput["input_ids"].index(0)
    except ValueError:
        sent_len = len(tinput["input_ids"])
    # Start at 1 to skip pred for cls token
    i = 1

    for word in tinput['subwords']:
        
        word_score = np.zeros_like(pred[0])
        eos_reached = False
        try:
            for subword in word:
                word_score += pred[i]
                i += 1
            label = labels.int2str(int(np.argmax(word_score)))
            pred_word_labels.append(label)
        except IndexError:
            pred_word_labels.append("O")

    for token, label in zip(tinput['words'], pred_word_labels):
        outfile.write("\n" + f"{token}\t{label}\t" + ('\t'.join(['_'] * 8)))

outfile.close()

In [ ]:
accuracy = datasets.load_metric("accuracy")
f1_metric = datasets.load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    refs = labels.flatten()
    predictions = np.argmax(logits, axis=-1)
    print(logits.shape)
    print(predictions.shape)
    pred = predictions.flatten()
    return {
        "accuracy": accuracy.compute(predictions=pred, references=refs)["accuracy"],
        "f1_micro": f1_metric.compute(predictions=pred, references=refs, average="micro")["f1"],
        "f1_macro": f1_metric.compute(predictions=pred, references=refs, average="macro")["f1"],
    }

In [ ]:
Trainer(
    model=model,
    args=training_args,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics,
).evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner, words, subwords, id. If ner, words, subwords, id are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1244
  Batch size = 8


(1244, 100, 12)
(1244, 100)


{'eval_accuracy': 0.9911173633440514,
 'eval_f1_macro': 0.7953250414897775,
 'eval_f1_micro': 0.9911173633440514,
 'eval_loss': 0.03592026233673096,
 'eval_runtime': 19.0616,
 'eval_samples_per_second': 65.262,
 'eval_steps_per_second': 8.184}